# Effects of  attention on Covid analysis

In this notebook, we wish to show the pipline we will use to explore the effects that increased attention on Covid in a coutry had on the percentage of people infected and died as a consequence of Covid in this country. The piplne in this notebook includes:
1. Computing the `covid attention score` for each country
2. Create a new dataset describing the county, and its policies and general stuation during the first wave of the pandemic
3. Add Covid's outcome (number of infections and deaths) after the end of the first wave for each country in this dataset
4. Compute the propensity score for each country
5. Balancing the dataset via matching

The fundamental idea of this taks is to explore effecets of the overall interest about Covid and its presence in the social media topics (e.g., twitter) in the period that preceeded the offical lockdowns (or any other similar measure, e.g, school closure, for countries that did not impose the lockdowns) on the resulting percetage of people infected and died for each country.

It is important to note why we focus on the exact period before the lockdowns (or other significant measures): during this period Covid has spread mostly accross countries (afterwards lockdows mitgated its spread), so we want to inspect how did influential people's and general public's attention on Covid help (or not?) contry fight pandemic when, in essence, this was the only way to set people behavior, e.g., staying at home, avoiding social contacts, before this behvaior was imposed by governments. 


For the purposes of the second project Milestone, we display how pipline work on just two countries: Serbia and Italy. We will extend this analysis for various of countries for the final Milestone. 

In [1]:
import pandas as pd
import numpy as np

import re
import json
import string
from datetime import datetime, timedelta

import translators as ts

import networkx as nx
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Natural language processing libraries
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Twitter library
import tweepy

# Data visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Topic analysis libraries
import pyLDAvis 
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
import pyLDAvis.gensim_models as gensimvi

import gensim
from gensim import corpora
import pickle
import bz2
import json

import warnings
warnings.filterwarnings("ignore")

import spacy
%matplotlib inline

Using state Vaud server backend.
C:\Users\Korisnik\anaconda3\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
C:\Users\Korisnik\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


## 1. Covid attention score

In order to compute the `covid attention score` for each country, we can rely on two sources:
1. Twitter data of influential people - represents the presence of COVID in the media
2. Wikipedia pageviews - represents the people's general intrest in Covid

We gather this data for each country for the period of 3 weeks before the offical lockdown imposed by government. Then for twitter data we utilize the LDA topic model to pick up the key topics talked about by influencial people on twitter. Thus, we can find the topic that is related to pandemics, covid, lockdown, and so on, and find the percetange of tokens related to it. This percetage is our score for  presence of COVID on the socal media. Next, we use the `aggregated_timeseries.json` dataset to get the average percetage of pageviews going to covid-related pages per day for pages in the appropriate langauge for the country we analyze.

Finally, we average these two results by Harmonic mean to obtain the Covid attention score. We use the Harmonic mean to more evenly weight the influence of each of the percetage score we use. For example, $1\%$ pageviews to covid-related pages represents a very significant interest to Covid compared to the number of topics available on wikipedia. On the other hand, number of topics we can explore by topic analysis is orders of magnitued smaller, so signifcant attention to Covid is characterized by larger percetages. 

In [2]:
# twitter data for influential people
influential_people_tweets = pd.read_csv('output/influential_people_tweets.csv', delimiter=',')
influential_people_tweets.head()

,Unnamed: 0,id,country_code,lang,user,tweet_text_orginal,tweet_text_en,tweet_date,context_annotations
0,0,1240013952861511680,RS,sr,Response(data=<User id=356450858 name=Александ...,Поносни смо на наше пријатељство.\r\nНикада не...,we are proud of our friendship we will never f...,2020-03-17 20:35:39+00:00,[]
1,1,1239873649999523845,RS,sr,Response(data=<User id=356450858 name=Александ...,Бескрајно хвала на свему нашој кинеској браћи ...,infinitely thank you for all our chinese broth...,2020-03-17 11:18:08+00:00,"[{'domain': {'id': '123', 'name': 'Ongoing New..."
2,2,1239310408760074240,RS,sr,Response(data=<User id=356450858 name=Александ...,"Предаја није, никада није била и никада неће б...",the surrender is not it was never and will nev...,2020-03-15 22:00:01+00:00,[]
3,3,1238813645385187328,RS,sr,Response(data=<User id=356450858 name=Александ...,"Pадимо, боримо се и урадићемо све што треба. С...",we fall we fight and we will do whatever you n...,2020-03-14 13:06:03+00:00,[]
4,4,1237796648161599491,RS,sr,Response(data=<User id=356450858 name=Александ...,"Одлуке доноси струка, не политика.\r\nХвала на...",decisions make a profession not politics thank...,2020-03-11 17:44:52+00:00,[]


In [3]:
# json data of aggregated time-series of wikipedia pageviews
json_file_path = "data/aggregated_timeseries.json"
with open(json_file_path, 'r') as j:
     aggregated_timeseries = json.loads(j.read())

In [4]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

### Covid attention score Italy

In [5]:
# Here we compute the percentage of tokens in Covid related topic for Italy
country_code = 'IT'
tweets_from_country = influential_people_tweets.loc[influential_people_tweets['country_code']==country_code, 
                                                        'tweet_text_en']

data_words = []
for row in tweets_from_country:
    data_words.append(str(row).split())
    
print("Words example\n", data_words[:1])

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print("\nTrigram example\n", trigram_mod[bigram_mod[data_words[0]]])

# Initialize spaCy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

data_words_bigrams = make_bigrams(data_words)

# Perform lemmatization keeping noun, adjective, verb, and adverb
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN']) #, 'ADJ', 'VERB', 'ADV'

print("\nData lemmatization\n", data_lemmatized[:1])

id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# Create a LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=5,
                                           passes=10,
                                           alpha='auto',
                                           eval_every=5, 
                                           per_word_topics=True)

doc_lda = lda_model[corpus]

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

pyLDAvis.enable_notebook()
visualization = gensimvi.prepare(lda_model, corpus, id2word)
visualization

Words example
 [['well', 'premier', 'conte', 'on', 'the', 'fact', 'of', 'considering', 'all', 'of', 'italy', '#zonarossa', 'also', 'good', 'on', 'the', 'commissioner', 'for', 'the', 'emergency', 'now', 'help', 'to', 'families', 'and', 'businesses', 'are', 'needed', 'liquidity', 'for', 'all', '#stopmutui', 'we', 'will', 'win', 'together', 'the', 'challenge', 'of', 'the', '#coronavirus']]

Trigram example
 ['well', 'premier', 'conte', 'on', 'the', 'fact', 'of', 'considering', 'all', 'of', 'italy', '#zonarossa', 'also', 'good', 'on', 'the', 'commissioner', 'for', 'the', 'emergency', 'now', 'help', 'to', 'families', 'and', 'businesses', 'are', 'needed', 'liquidity', 'for', 'all', '#stopmutui', 'we', 'will', 'win', 'together', 'the', 'challenge', 'of', 'the', '#coronavirus']

Data lemmatization
 [['fact', 'commissioner', 'emergency', 'family', 'business', 'liquidity', '#', 'stopmutui', 'challenge', 'coronavirus']]

Perplexity:  -9.140127917688593


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.075820 -0.027765       1        1  31.754331
0     -0.001698  0.038637       2        1  28.378696
2     -0.067895 -0.034102       3        1  20.591077
1     -0.006227  0.023231       4        1  19.275896, topic_info=        Term       Freq      Total Category  logprob  loglift
218     life  99.000000  99.000000  Default  30.0000  30.0000
137     time  61.000000  61.000000  Default  29.0000  29.0000
143      day  37.000000  37.000000  Default  28.0000  28.0000
0          #  39.000000  39.000000  Default  27.0000  27.0000
19     today  28.000000  28.000000  Default  26.0000  26.0000
...      ...        ...        ...      ...      ...      ...
412     care   4.613702  25.134177   Topic4  -4.9722  -0.0489
1147   grace   4.048547  20.549278   Topic4  -5.1029   0.0218
1148   mercy   3.678876  19.390397   Topic4  -5.1987  -0.0159
1178  gospel   3.210421  15.746308   Topic4  -5.3349   0.0561
1098  memory   3.307484  18.646978   Topic4  -5.3051  -0.0832

[294 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.977842         #
1092      1  0.398406  adhesion
1092      2  0.398406  adhesion
1092      3  0.398406  adhesion
1092      4  0.398406  adhesion
...     ...       ...       ...
231       1  0.710368    worker
231       2  0.142074    worker
231       4  0.142074    worker
53        1  0.949862     world
54        2  0.970014      year

[374 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 3, 2])

It is clear that the topic one is related to the covid and it contains 31.8% of tokens.

Let us now find the average percetage of pageviews going to covid-related pages per day for pages for Italy.

In [6]:
dates = [date for date in pd.to_datetime(
    list(aggregated_timeseries["it.m"]["covid"]["sum"].keys()))  if date >= datetime(2020, 2, 17)]

dates = [date for date in dates if date <= datetime(2020, 3, 10)]

In [7]:
covid_sr = pd.Series(
    [(aggregated_timeseries["it.m"]["covid"]["sum"][str(key)] + aggregated_timeseries["it"]["covid"]["sum"][str(key)]) / (
        aggregated_timeseries["it.m"]["sum"][str(key)] + aggregated_timeseries["it"]["sum"][str(key)])
                      for key in dates])
covid_sr

0     0.000299
1     0.000296
2     0.000252
3     0.000243
4     0.001600
5     0.003682
6     0.006197
7     0.005384
8     0.003242
9     0.002705
10    0.002135
11    0.001722
12    0.001287
13    0.001040
14    0.001181
15    0.001370
16    0.002126
17    0.001941
18    0.002203
19    0.002517
20    0.003451
21    0.003996
22    0.004280
dtype: float64

In [8]:
covid_sr.describe()

count    23.000000
mean      0.002311
std       0.001624
min       0.000243
25%       0.001234
50%       0.002126
75%       0.003347
max       0.006197
dtype: float64

In [9]:
print("Covid attention score for Italy", 2 * 0.002311 * 0.318 / (0.002311 + 0.318))

Covid attention score for Italy 0.0045886529029599355


In [10]:
attention = {"Italy": 2 * 0.002311 * 0.318 / (0.002311 + 0.318) *  1000}

We multiply attention score by 1000, so we have a number that is nicer to see.

### Covid attention score Serbia

In [11]:
# Here we compute the percentage of tokens in Covid related topic for Italy
country_code = 'RS'
tweets_from_country = influential_people_tweets.loc[influential_people_tweets['country_code']==country_code, 
                                                        'tweet_text_en']

data_words = []
for row in tweets_from_country:
    data_words.append(str(row).split())
    
print("Words example\n", data_words[:1])

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print("\nTrigram example\n", trigram_mod[bigram_mod[data_words[0]]])

# Initialize spaCy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

data_words_bigrams = make_bigrams(data_words)

# Perform lemmatization keeping noun, adjective, verb, and adverb
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN']) #, 'ADJ', 'VERB', 'ADV'

print("\nData lemmatization\n", data_lemmatized[:1])

id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# Create a LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=5,
                                           passes=10,
                                           alpha='auto',
                                           eval_every=5, 
                                           per_word_topics=True)

doc_lda = lda_model[corpus]

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

pyLDAvis.enable_notebook()
visualization = gensimvi.prepare(lda_model, corpus, id2word)
visualization

Words example
 [['we', 'are', 'proud', 'of', 'our', 'friendship', 'we', 'will', 'never', 'forget', 'the', 'help', 'of', 'our', 'chinese', 'friends']]

Trigram example
 ['we', 'are', 'proud', 'of', 'our', 'friendship', 'we', 'will', 'never', 'forget', 'the', 'help', 'of', 'our', 'chinese', 'friends']

Data lemmatization
 [['friendship', 'help', 'friend']]

Perplexity:  -9.40173027378055


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.165759  0.015876       1        1  24.817623
5      0.061360  0.095585       2        1  18.509373
4      0.029215 -0.081795       3        1  16.549255
3      0.023557 -0.003046       4        1  14.739442
0      0.041461 -0.016024       5        1  14.144599
2      0.010166 -0.010596       6        1  11.239708, topic_info=          Term       Freq      Total Category  logprob  loglift
628      debil  29.000000  29.000000  Default  30.0000  30.0000
630      moron  10.000000  10.000000  Default  29.0000  29.0000
629       bass  10.000000  10.000000  Default  28.0000  28.0000
227     parent   8.000000   8.000000  Default  27.0000  27.0000
37      people  13.000000  13.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
604  pineapple   0.585776   5.635120   Topic6  -5.4300  -0.0781
605   verenica   0.585776   5.635120   Topic6  -5.4300  -0.0781
601      apple   0.390924   3.413776   Topic6  -5.8344   0.0187
602       base   0.390924   3.413776   Topic6  -5.8344   0.0187
603        min   0.390924   3.413776   Topic6  -5.8344   0.0187

[228 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
8         2  0.631023       #
617       1  0.825559     air
622       1  0.851399   album
601       5  0.585862   apple
623       1  0.851399   balad
...     ...       ...     ...
487       2  0.596150     way
85        3  0.792774    week
128       2  0.526601    word
616       1  0.825557  writer
31        1  0.864605    year

[152 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 6, 5, 4, 1, 3])

It is clear that the topic one is related to the covid and it contains 16.5% of tokens.

Let us now find the average percetage of pageviews going to covid-related pages per day for pages for Serbia.

In [12]:
dates = [date for date in pd.to_datetime(
    list(aggregated_timeseries["sr.m"]["covid"]["sum"].keys()))  if date >= datetime(2020, 3, 1)]

dates = [date for date in dates if date <= datetime(2020, 3, 20)]

In [13]:
covid_sr = pd.Series(
    [(aggregated_timeseries["sr.m"]["covid"]["sum"][str(key)] + aggregated_timeseries["sr"]["covid"]["sum"][str(key)]) / (
        aggregated_timeseries["sr.m"]["sum"][str(key)] + aggregated_timeseries["sr"]["sum"][str(key)])
                      for key in dates])
covid_sr

0     0.000024
1     0.000034
2     0.000037
3     0.000033
4     0.000040
5     0.000050
6     0.000042
7     0.000057
8     0.000040
9     0.000128
10    0.000501
11    0.001344
12    0.001607
13    0.001691
14    0.002050
15    0.002299
16    0.001999
17    0.002128
18    0.001702
19    0.001441
dtype: float64

In [14]:
covid_sr.describe()

count    20.000000
mean      0.000862
std       0.000907
min       0.000024
25%       0.000040
50%       0.000314
75%       0.001693
max       0.002299
dtype: float64

In [15]:
print("Covid attention score for Serbia", 2 * 0.000862 * 0.165 / (0.000862 + 0.165))

Covid attention score for Serbia 0.0017150402141539352


In [16]:
attention["Serbia"] = 2 * 0.000862 * 0.165 / (0.000862 + 0.165) * 1000

## 2. Countries dataset

In [17]:
applemobilitytrends = pd.read_csv("data/applemobilitytrends-2020-04-20.csv.gz")
applemobilitytrends.sample(5) 

,geo_type,region,transportation_type,2020-01-13,2020-01-14,2020-01-15,2020-01-16,2020-01-17,2020-01-18,2020-01-19,...,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20
272,city,Lyon,transit,100,104.18,106.60,101.66,121.07,115.58,93.40,...,25.39,21.21,26.29,30.57,32.54,31.12,30.22,30.40,28.53,33.37
322,city,Perth,transit,100,103.54,103.72,99.56,95.28,85.57,98.78,...,15.48,17.22,23.88,23.23,23.47,24.53,20.02,15.60,21.16,23.25
236,city,Guadalajara,walking,100,106.53,108.05,113.21,125.74,127.41,84.53,...,30.07,23.85,38.34,39.84,40.24,39.37,43.38,40.66,25.62,36.72
264,city,Lille,walking,100,109.71,112.18,137.62,138.07,170.12,76.16,...,17.95,14.07,14.56,19.18,19.44,21.16,18.58,17.73,17.47,21.86
205,city,Copenhagen,driving,100,103.18,106.19,104.64,109.86,101.87,95.48,...,56.86,55.60,59.64,68.18,71.48,72.90,74.28,67.22,70.76,76.53


In [18]:
mobility = applemobilitytrends.groupby("region").mean()
mobility = mobility[(mobility.index == "Serbia") | (mobility.index == "Italy")].iloc[:, :58]
mobility

,2020-01-13,2020-01-14,2020-01-15,2020-01-16,2020-01-17,2020-01-18,2020-01-19,2020-01-20,2020-01-21,2020-01-22,...,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10
region,,,,,,,,,,,,,,,,,,,,,
Italy,100.0,101.850,103.556667,104.340,114.916667,127.72,109.463333,101.306667,101.746667,103.970,...,72.69,70.910,73.910,72.713333,69.483333,71.310,70.73,55.766667,44.13,29.913333
Serbia,100.0,91.365,97.195000,97.145,107.460000,110.93,87.475000,90.935000,94.990000,95.445,...,103.09,97.965,90.575,94.740000,99.070000,108.985,116.39,88.500000,94.74,87.770000


In [19]:
countries = pd.read_csv("data/countries.csv", thousands=',')
countries = countries[(countries["Country"] == "Serbia ") | (
    countries["Country"] == "Italy ")].dropna(axis=1, how='any')

countries["Country"] = countries["Country"].apply(lambda x : x.strip())
countries.drop("Region", axis=1, inplace=True)
countries

,Country,Population,Area,Pop_Density,Coastline,Net_migration,Infant_mortality,GDP_per_capita,Literacy,Phones_per_1000,Arable,Crops,Other,Agriculture,Industry,Service
101,Italy,58133509,301230,1930,252,207.0,594.0,26700.0,986.0,4309.0,2779.0,953.0,6268.0,21.0,291.0,688.0
181,Serbia,9396411,88361,1063,0,-133.0,1289.0,2200.0,930.0,2858.0,3335.0,32.0,6345.0,166.0,255.0,579.0


In [20]:
countries = countries.merge(mobility, left_on="Country", right_index=True)
countries

,Country,Population,Area,Pop_Density,Coastline,Net_migration,Infant_mortality,GDP_per_capita,Literacy,Phones_per_1000,...,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10
101,Italy,58133509,301230,1930,252,207.0,594.0,26700.0,986.0,4309.0,...,72.69,70.910,73.910,72.713333,69.483333,71.310,70.73,55.766667,44.13,29.913333
181,Serbia,9396411,88361,1063,0,-133.0,1289.0,2200.0,930.0,2858.0,...,103.09,97.965,90.575,94.740000,99.070000,108.985,116.39,88.500000,94.74,87.770000


In [21]:
global_mobility = pd.read_csv("data/Global_Mobility_Report.csv")
global_mobility = global_mobility.groupby("country_region").mean()

global_mobility = global_mobility[(global_mobility.index == "Serbia") | (global_mobility.index == "Italy")].dropna(axis=1, how='any')

In [22]:
countries = countries.merge(global_mobility, left_on="Country", right_index=True)
countries

,Country,Population,Area,Pop_Density,Coastline,Net_migration,Infant_mortality,GDP_per_capita,Literacy,Phones_per_1000,...,2020-03-07,2020-03-08,2020-03-09,2020-03-10,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
101,Italy,58133509,301230,1930,252,207.0,594.0,26700.0,986.0,4309.0,...,70.73,55.766667,44.13,29.913333,-32.069112,-18.387409,28.121185,-33.729003,-32.788422,11.221439
181,Serbia,9396411,88361,1063,0,-133.0,1289.0,2200.0,930.0,2858.0,...,116.39,88.500000,94.74,87.770000,-28.793919,-14.459459,-2.270270,-35.050676,-34.091216,8.614865


In [23]:
interventions = pd.read_csv("data/interventions.csv")
interventions

,lang,1st case,1st death,School closure,Public events banned,Lockdown,Mobility,Normalcy
0,fr,2020-01-24,2020-02-14,2020-03-14,2020-03-13,2020-03-17,2020-03-16,2020-07-02
1,da,2020-02-27,2020-03-12,2020-03-13,2020-03-12,2020-03-18,2020-03-11,2020-06-05
2,de,2020-01-27,2020-03-09,2020-03-14,2020-03-22,2020-03-22,2020-03-16,2020-07-10
3,it,2020-01-31,2020-02-22,2020-03-05,2020-03-09,2020-03-11,2020-03-11,2020-06-26
4,nl,2020-02-27,2020-03-06,2020-03-11,2020-03-24,NaN,2020-03-16,2020-05-29
5,no,2020-02-26,2020-02-26,2020-03-13,2020-03-12,2020-03-24,2020-03-11,2020-06-04
6,sr,2020-03-06,2020-03-20,2020-03-15,2020-03-21,2020-03-21,2020-03-16,2020-05-02
7,sv,2020-01-31,2020-03-11,2020-03-18,2020-03-12,NaN,2020-03-11,2020-06-05
8,ko,2020-01-20,2020-02-20,2020-02-23,NaN,NaN,2020-02-25,2020-04-15
9,ca,2020-01-31,2020-02-13,2020-03-12,2020-03-08,2020-03-14,2020-03-16,NaN


In [24]:
interventions["Had lockdown"] = interventions["Lockdown"].apply(lambda x : 1 if isinstance(x, str) else 0)
interventions[["lang", "Had lockdown"]]

,lang,Had lockdown
0,fr,1
1,da,1
2,de,1
3,it,1
4,nl,0
5,no,1
6,sr,1
7,sv,0
8,ko,0
9,ca,1


In [25]:
interventions = interventions[(interventions["lang"] == "it") | (interventions["lang"] == "sr")][["lang", "Had lockdown"]]
interventions

,lang,Had lockdown
3,it,1
6,sr,1


In [26]:
interventions["Had lockdown"].values

array([1, 1], dtype=int64)

In [27]:
countries["Lockdown"] = interventions["Had lockdown"].values
countries

,Country,Population,Area,Pop_Density,Coastline,Net_migration,Infant_mortality,GDP_per_capita,Literacy,Phones_per_1000,...,2020-03-08,2020-03-09,2020-03-10,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,Lockdown
101,Italy,58133509,301230,1930,252,207.0,594.0,26700.0,986.0,4309.0,...,55.766667,44.13,29.913333,-32.069112,-18.387409,28.121185,-33.729003,-32.788422,11.221439,1
181,Serbia,9396411,88361,1063,0,-133.0,1289.0,2200.0,930.0,2858.0,...,88.500000,94.74,87.770000,-28.793919,-14.459459,-2.270270,-35.050676,-34.091216,8.614865,1


## 3. Add Covid's outcome

In [28]:
covid_per_country = pd.read_csv("data/WHO-COVID-19-global-data.csv")
covid_per_country.head(5)

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-03,AF,Afghanistan,EMRO,0,0,0,0
1,2020-01-04,AF,Afghanistan,EMRO,0,0,0,0
2,2020-01-05,AF,Afghanistan,EMRO,0,0,0,0
3,2020-01-06,AF,Afghanistan,EMRO,0,0,0,0
4,2020-01-07,AF,Afghanistan,EMRO,0,0,0,0


In [29]:
covid_per_country[covid_per_country["Country"] == "Serbia"].iloc[-1]

Date_reported        2022-10-07
Country_code                 RS
Country                  Serbia
WHO_region                 EURO
New_cases                  1872
Cumulative_cases        2373346
New_deaths                    7
Cumulative_deaths         17057
Name: 192718, dtype: object

In [30]:
covid_per_country = covid_per_country[(covid_per_country["Country"] == "Serbia") | (covid_per_country["Country"] == "Italy")].groupby(
    "Country").apply(lambda x : x[x["Date_reported"] == "2020-05-15"][["Date_reported", "Cumulative_cases", "Cumulative_deaths"]])

covid_per_country

,,Date_reported,Cumulative_cases,Cumulative_deaths
Country,,,,
Italy,104060,2020-05-15,223096,31368
Serbia,191843,2020-05-15,10374,224


In [31]:
covid_per_country["Country"] = [idx[0] for idx in covid_per_country.index]
covid_per_country

,,Date_reported,Cumulative_cases,Cumulative_deaths,Country
Country,,,,,
Italy,104060,2020-05-15,223096,31368,Italy
Serbia,191843,2020-05-15,10374,224,Serbia


In [32]:
covid_per_country = covid_per_country.set_index("Country")
covid_per_country.drop("Date_reported", axis=1, inplace=True)
covid_per_country

,Cumulative_cases,Cumulative_deaths
Country,,
Italy,223096,31368
Serbia,10374,224


In [33]:
countries = countries.merge(covid_per_country, left_on="Country", right_index=True)
countries

,Country,Population,Area,Pop_Density,Coastline,Net_migration,Infant_mortality,GDP_per_capita,Literacy,Phones_per_1000,...,2020-03-10,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,Lockdown,Cumulative_cases,Cumulative_deaths
101,Italy,58133509,301230,1930,252,207.0,594.0,26700.0,986.0,4309.0,...,29.913333,-32.069112,-18.387409,28.121185,-33.729003,-32.788422,11.221439,1,223096,31368
181,Serbia,9396411,88361,1063,0,-133.0,1289.0,2200.0,930.0,2858.0,...,87.770000,-28.793919,-14.459459,-2.270270,-35.050676,-34.091216,8.614865,1,10374,224


In [34]:
countries["Cumulative_cases"] /= countries["Population"]
countries["Cumulative_deaths"] /= countries["Population"]
countries.set_index("Country", inplace=True)
countries

,Population,Area,Pop_Density,Coastline,Net_migration,Infant_mortality,GDP_per_capita,Literacy,Phones_per_1000,Arable,...,2020-03-10,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,Lockdown,Cumulative_cases,Cumulative_deaths
Country,,,,,,,,,,,,,,,,,,,,,
Italy,58133509,301230,1930,252,207.0,594.0,26700.0,986.0,4309.0,2779.0,...,29.913333,-32.069112,-18.387409,28.121185,-33.729003,-32.788422,11.221439,1,0.003838,0.000540
Serbia,9396411,88361,1063,0,-133.0,1289.0,2200.0,930.0,2858.0,3335.0,...,87.770000,-28.793919,-14.459459,-2.270270,-35.050676,-34.091216,8.614865,1,0.001104,0.000024


## 4. Compute the propensity score for each country

In [35]:
attention

{'Italy': 4.588652902959936, 'Serbia': 1.7150402141539351}

In [36]:
countries["Attention"] = [1, 0]

In [37]:
countries

,Population,Area,Pop_Density,Coastline,Net_migration,Infant_mortality,GDP_per_capita,Literacy,Phones_per_1000,Arable,...,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,Lockdown,Cumulative_cases,Cumulative_deaths,Attention
Country,,,,,,,,,,,,,,,,,,,,,
Italy,58133509,301230,1930,252,207.0,594.0,26700.0,986.0,4309.0,2779.0,...,-32.069112,-18.387409,28.121185,-33.729003,-32.788422,11.221439,1,0.003838,0.000540,1
Serbia,9396411,88361,1063,0,-133.0,1289.0,2200.0,930.0,2858.0,3335.0,...,-28.793919,-14.459459,-2.270270,-35.050676,-34.091216,8.614865,1,0.001104,0.000024,0


In [ ]:
for column in countries.columns[:-3]:
    countries[column] = (countries[column] - countries[column].mean()) / (1 if countries[column].std() == 0 else countries[column].std())
    
mod = smf.logit(formula="Attention ~  Population + Area + Pop_Density + Coastline + Net_migration +\
       Infant_mortality + GDP_per_capita + Literacy + Phones_per_1000 +\
       Arable + Crops + Other + Agriculture + Industry + Service +\
       '2020-01-13' + '2020-01-14' + '2020-01-15' + '2020-01-16' + '2020-01-17' +\
       '2020-01-18' + '2020-01-19' + '2020-01-20' + '2020-01-21' + '2020-01-22' +\
       '2020-01-23' + '2020-01-24' + '2020-01-25' + '2020-01-26' + '2020-01-27' +\
       '2020-01-28' + '2020-01-29' + '2020-01-30' + '2020-01-31' + '2020-02-01' +\
       '2020-02-02' + '2020-02-03' + '2020-02-04' + '2020-02-05' + '2020-02-06' +\
       '2020-02-07' + '2020-02-08' + '2020-02-09' + '2020-02-10' + '2020-02-11' +\
       '2020-02-12' + '2020-02-13' + '2020-02-14' + '2020-02-15' + '2020-02-16' +\
       '2020-02-17' + '2020-02-18' + '2020-02-19' + '2020-02-20' + '2020-02-21' +\
       '2020-02-22' + '2020-02-23' + '2020-02-24' + '2020-02-25' + '2020-02-26' +\
       '2020-02-27' + '2020-02-28' + '2020-02-29' + '2020-03-01' + '2020-03-02' +\
       '2020-03-03' + '2020-03-04' + '2020-03-05' + '2020-03-06' + '2020-03-07' +\
       '2020-03-08' + '2020-03-09' + '2020-03-10' +\
       retail_and_recreation_percent_change_from_baseline + grocery_and_pharmacy_percent_change_from_baseline +\
       parks_percent_change_from_baseline + transit_stations_percent_change_from_baseline +\
       workplaces_percent_change_from_baseline + residential_percent_change_from_baseline +\
       C(Lockdown) ", data=countries)

res = mod.fit()

# Extract the estimated propensity scores
countries['Propensity_score'] = res.predict()

print(res.summary())

**PerfectSeparationError:** Perfect separation detected, results not available

## 5. Balancing the dataset via matching

In [ ]:
# Separate the treatment and control groups
treatment_df = countries[countries['treat'] == 1]
control_df = countries[countries['treat'] == 0]

# Create an empty undirected graph
G = nx.Graph()

# Loop through all the pairs of instances
for control_id, control_row in control_df.iterrows():
    for treatment_id, treatment_row in treatment_df.iterrows():

        # Calculate the similarity 
        similarity = get_similarity(control_row['Propensity_score'],
                                    treatment_row['Propensity_score'])

        # Add an edge between the two instances weighted by the similarity between them
        G.add_weighted_edges_from([(control_id, treatment_id, similarity)])

# Generate and return the maximum weight matching on the generated graph
matching = nx.max_weight_matching(G)

matched = [i[0] for i in list(matching)] + [i[1] for i in list(matching)]
countries_balanced = countries.iloc[matched]